In [1]:
### Q2.
## Load packages 
using JuMP, HiGHS, Random, Distributions

## Define the data
T = 14
r_employees = 1:10
r_employees_names = [:A, :B, :C, :D, :E, :F, :G, :H, :I, :J]
r_salary = [135, 150, 165, 150, 165, 135, 165, 150, 135, 150]
r_prod = [25, 30, 35, 30, 35, 25, 35, 30, 25, 30]

p_salary = 40
p_prod = 10

## Define function for solving employees scheduling problem
function solve_esp(demandscens)

    # Calculate the number of given scenarios
    numscen = size(demandscens, 2)
    scenarios = 1:numscen
    
    # Create the model
    m = Model()

    # Declare all the decision variables
    @variable(m, s_regular[r_employees, 1:T], Bin)
    @variable(m, s_part[1:T, scenarios] >= 0)

    # Define the objective
    @objective(m, Min, sum(r_salary[i]*s_regular[i,t] for i in r_employees, t in 1:T)
        + 1/numscen*sum(p_salary*s_part[t,s] for t in 1:T, s in scenarios))

    # Write all the constraints
    @constraint(m, s_meetdemand[t in 1:T, s in scenarios],
        p_prod*s_part[t,s] + sum(r_prod[i]*s_regular[i,t] for i in r_employees) >= demandscens[t,s])

    @constraint(m, s_regular_day_lhs[i in r_employees, w in 1:div(T,7)],
        4 <= sum(s_regular[i,t] for t in 7*w-6:7*w))

    @constraint(m, s_regular_day_rhs[i in r_employees, w in 1:div(T,7)],
        sum(s_regular[i,t] for t in 7*w-6:7*w) <= 6)

    @constraint(m, s_regular_conti[i in r_employees, d in 1:T-3],
        sum(s_regular[i,t] for t in d:d+3) <= 3)

    # Set optimizer and solve the model
    set_optimizer(m, HiGHS.Optimizer)
    set_silent(m)
    optimize!(m)

    # Print Solution time and expected cost
    println("Solution time: ", solve_time(m))
    println("Optimal value: ", objective_value(m))

    # Return objective value and x_values for problem 4
    return objective_value(m), value.(s_regular)
end

solve_esp (generic function with 1 method)

In [2]:
### Q2. Small sample size

## Set seed
Random.seed!(502)

## Define sample size
numscen_1 = 100
scenarios_1 = 1:numscen_1

## Generate random numbers for scenarios
demandscens_1 = Array{Float64}(undef,T,numscen_1)
for s in scenarios_1
    for t in 1:T
        demandscens_1[t,s] = round(rand(Normal(225,20)),digits=0)
    end
end

## Solve the problem
solve_esp(demandscens_1)

Solution time: 1.9818439483642578
Optimal value: 15039.04


(15039.04, 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, Base.OneTo(14)
And data, a 10×14 Matrix{Float64}:
  1.0                 0.0  1.0  1.0  -0.0                 …  0.0  1.0  -0.0
  1.000000000000003  -0.0  0.0  1.0   1.0                    1.0  0.0   1.0
  1.0                 0.0  1.0  1.0   1.0                    1.0  0.0   0.0
  0.0                 0.0  1.0  1.0  -0.0                    1.0  0.0   1.0
  1.0                 1.0  0.0  0.0   1.0                    1.0  0.0  -0.0
  0.0                 0.0  1.0  1.0   0.9999999999999993  …  0.0  1.0   1.0
  1.0                 1.0  0.0  0.0   0.0                    1.0  1.0   1.0
  0.0                 1.0  0.0  0.0   1.0                    0.0  1.0   0.0
  0.0                 1.0  1.0  1.0   0.0                    0.0  1.0   1.0
 -0.0                 1.0  1.0  0.0   1.0                    0.0  1.0   1.0)

In [3]:
### Q2. Medium sample size

## Set seed
Random.seed!(502)

## Define sample size
numscen_2 = 200
scenarios_2 = 1:numscen_2

## Generate random numbers for scenarios
demandscens_2 = Array{Float64}(undef,T,numscen_2)
for s in scenarios_2
    for t in 1:T
        demandscens_2[t,s] = round(rand(Normal(225,20)),digits=0)
    end
end

## Solve the problem
solve_esp(demandscens_2)

Solution time: 2.733991861343384
Optimal value: 14997.599999999999


(14997.599999999999, 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, Base.OneTo(14)
And data, a 10×14 Matrix{Float64}:
 1.0  0.0   0.9999999999999979  1.0  …   0.0
 0.0  1.0   0.0                 1.0      1.0
 1.0  1.0   0.0                 0.0      1.0
 1.0  0.0  -0.0                 0.0      1.0
 0.0  1.0   1.0                 1.0      1.0
 1.0  0.0   1.0                 1.0  …   0.0
 0.0  1.0   1.0                 0.0      0.0
 1.0  0.0   0.0                 1.0      1.0
 1.0  0.0   1.0000000000000016  1.0      0.0
 0.0  1.0   1.0                 0.0     -9.239311256105938e-15)

In [4]:
### Q2. Large sample size

## Set seed
Random.seed!(502)

## Define sample size
numscen_3 = 300
scenarios_3 = 1:numscen_3

## Generate random numbers for scenarios
demandscens_3 = Array{Float64}(undef,T,numscen_3)
for s in scenarios_3
    for t in 1:T
        demandscens_3[t,s] = round(rand(Normal(225,20)),digits=0)
    end
end

## Solve the problem
solve_esp(demandscens_3)

Solution time: 6.247933864593506
Optimal value: 15008.09333333332


(15008.09333333332, 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, Base.OneTo(14)
And data, a 10×14 Matrix{Float64}:
 0.0  1.0  -0.0  1.0  -0.0  1.0  1.0  0.0  …   1.0                     1.0
 1.0  0.0   1.0  1.0   0.0  0.0  1.0  0.0     -0.0                     1.0
 1.0  1.0  -0.0  0.0   1.0  1.0  0.0  0.0     -2.582458038982745e-13   1.0
 1.0  0.0   0.0  1.0   1.0  0.0  1.0  0.0     -3.2187585929932536e-13  1.0
 1.0  1.0  -0.0  0.0   1.0  1.0  0.0  1.0      5.67323965583455e-14    0.0
 0.0  1.0   1.0  0.0   0.0  1.0  1.0  1.0  …   1.0                     1.0
 1.0  0.0   1.0  1.0  -0.0  0.0  1.0  1.0      1.0                     0.0
 0.0  1.0   1.0  0.0   1.0  1.0  0.0  1.0      1.0                     1.0
 0.0  1.0   1.0  1.0   0.0  1.0  0.0  1.0      1.0                     0.0
 0.0  0.0   1.0  1.0   1.0  0.0  1.0  1.0      1.0                     0.0)

In [5]:
### Q3. Risk Averse Model

## Define function for solving employees scheduling problem considering risk measures 
function solve_esp_risk(option)
    # Set risk parameter
    lambda = 1

    # Set seed
    Random.seed!(502) 

    # Define sample size
    numscen = 300
    scenarios = 1:numscen

    # Generate random numbers for scenarios
    demandscens=Array{Float64}(undef,T,numscen)
    for s in scenarios
        for t in 1:T
            demandscens[t,s]=round(rand(Normal(225,20)),digits=0)
        end
    end

    # Create the model
    m_msd = Model()

    # Declare all the decision variables
    @variable(m_msd, s_regular[r_employees, 1:T], Bin)
    @variable(m_msd, s_part[1:T, scenarios] >= 0)

    @variable(m_msd, s_re_cost[scenarios] >= 0)
    @variable(m_msd, s_mean_cost >= 0)
    @variable(m_msd, s_max[scenarios] >= 0)

    # Define the objective
    # If choose option 1, using the expected value as objective 
    if option == 1
        @objective(m_msd, Min, s_mean_cost)

    # If choose option 2, using the mean semi-deviation as objective
    elseif option == 2
        @objective(m_msd, Min, s_mean_cost + lambda*1/numscen*sum(s_max[s] for s in scenarios))
    end

    # Write all the constraints
    @constraint(m_msd, ex_avg[s in scenarios], s_max[s] >= s_re_cost[s] - s_mean_cost)

    @constraint(m_msd, record_cost[s in scenarios], s_re_cost[s] >= sum(r_salary[i]*s_regular[i,t] for i in r_employees, t in 1:T)
        + sum(p_salary*s_part[t,s] for t in 1:T))

    @constraint(m_msd, s_meetdemand[t in 1:T, s in scenarios],
        p_prod*s_part[t,s] + sum(r_prod[i]*s_regular[i,t] for i in r_employees) >= demandscens[t,s])

    @constraint(m_msd, s_regular_day_lhs[i in r_employees, w in 1:div(T,7)],
        4 <= sum(s_regular[i,t] for t in 7*w-6:7*w))

    @constraint(m_msd, s_regular_day_rhs[i in r_employees, w in 1:div(T,7)],
        sum(s_regular[i,t] for t in 7*w-6:7*w) <= 6)

    @constraint(m_msd, s_regular_conti[i in r_employees, d in 1:T-3],
        sum(s_regular[i,t] for t in d:d+3) <= 3)

    @constraint(m_msd, mean_cost, s_mean_cost == 1/numscen*sum(s_re_cost[s] for s in scenarios))

    # Set optimizer and solve the model
    set_optimizer(m_msd, HiGHS.Optimizer)
    set_silent(m_msd)
    optimize!(m_msd)

    # Print Solution time and expected cost
    println("Solution time: ", solve_time(m_msd))
    println("Optimal value: ", objective_value(m_msd))

    # Print the expected value of the objective
    println("The expected value of the objective: ", value.(s_mean_cost))

    # Print the risk measure of the objective
    println("The risk measure of the objective: ", sum(value.(s_max))/numscen)
end

solve_esp_risk (generic function with 1 method)

In [6]:
### Q3. Original objective
solve_esp_risk(1)

Solution time: 6.180672883987427
Optimal value: 15008.093333333321
The expected value of the objective: 15008.093333333321
The risk measure of the objective: 118.93999999999961


In [7]:
### Q3. Mean semi-deviation
solve_esp_risk(2)

Solution time: 97.62464118003845
Optimal value: 15126.587022222198
The expected value of the objective: 15007.973333333306
The risk measure of the objective: 118.6136888888926


In [8]:
### Q4. Sample Average Approximation

## Define the size of scenario, batch, evaluation set
numscen = 50  # n
numbatch = 15  # M
numeval = 500 # N

## First take one sample and solve to obtain a candidate solution
demandscens_saa=Array{Float64}(undef,T,numscen)
for s in 1:numscen
    for t in 1:T
        demandscens_saa[t,s]=round(rand(Normal(225,20)),digits=0)
    end
end

objval,candx = solve_esp(demandscens_saa)
println("candidate solution = (",  round.(candx,digits=4), ')')

## Define function for building and solving the second stage problem for a given value scenario and first-stage solution
## Returns the objective value
function solveSSP(xval,xival)
    yvals = zeros(Float64,T)
    for t in 1:T
        yvals[t] = max((xival[t]-sum(r_prod[i]*xval[i,t] for i in r_employees))/p_prod, 0.0)
    end
    
    return sum(r_salary[i]*xval[i,t] for i in r_employees, t in 1:T) + sum(p_salary*yvals[t] for t in 1:T)    
end

## Take a new sample to obtain an unbiased estimate of cost of the candidate solution and value of stochastic solution
evalvals = zeros(Float64,numeval)
evalscens = Array{Float64}(undef,T,numeval)
for s in 1:numeval
    for t in 1:T
        evalscens[t,s]=round(rand(Normal(225,20)),digits=0)
    end
end

for k in 1:numeval
    evalvals[k] = solveSSP(candx, evalscens[:,k])
end

ubmean = mean(evalvals)
ubwidth = std(evalvals)/sqrt(numeval)*2.145
println("ci on solution expected cost = [", round(ubmean-ubwidth,digits=3), ',', round(ubmean+ubwidth,digits=3), ']')

## Do the procedure to estimate an optimality gap
gapvals = zeros(Float64,numbatch)
for k in 1:numbatch
    # Set up and solve extensive form for a sample average approximation problem
    cursample = Array{Float64}(undef,T,numscen)
    for s in 1:numscen
        for t in 1:T
            cursample[t,s]=round(rand(Normal(225,20)),digits=0)
        end
    end

    sampleopt,curx = solve_esp(cursample) 

    # Evaluate the candidate solution using the same scenarios
    curvals = zeros(Float64,numscen)
    for s in 1:numscen
        curvals[s] = solveSSP(candx,cursample[:,s])
    end
    gapvals[k] = mean(curvals) - sampleopt 
end

println("mean gap estimate = ", round(mean(gapvals),digits=3))
println("95% c.i. on gap = [0, ", round(mean(gapvals) + std(gapvals)/sqrt(numbatch)*1.761,digits=3), ']')

Solution time: 0.8157787322998047
Optimal value: 14989.04
candidate solution = (2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, Base.OneTo(14)
And data, a 10×14 Matrix{Float64}:
 0.0  1.0  1.0  0.0  1.0  1.0  0.0   0.0   1.0   1.0   1.0  0.0  -0.0  1.0
 1.0  1.0  1.0  0.0  0.0  0.0  1.0  -0.0   1.0   0.0   1.0  1.0   0.0  1.0
 0.0  1.0  1.0  1.0  0.0  0.0  1.0   1.0   0.0   1.0  -0.0  0.0   1.0  1.0
 0.0  1.0  0.0  1.0  1.0  0.0  1.0   0.0   1.0   0.0   1.0  1.0   1.0  0.0
 1.0  0.0  1.0  0.0  1.0  1.0  0.0   1.0  -0.0  -0.0   1.0  1.0   1.0  0.0
 1.0  0.0  0.0  0.0  1.0  1.0  1.0   0.0   1.0   1.0   1.0  0.0  -0.0  1.0
 1.0  0.0  0.0  1.0  1.0  1.0  0.0   1.0   1.0   1.0   0.0  1.0   0.0  0.0
 0.0  1.0  1.0  0.0  0.0  1.0  1.0   1.0  -0.0   1.0   0.0  1.0   1.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  0.0   1.0   0.0   0.0   1.0  1.0   0.0  1.0
 1.0  1.0  0.0  1.0  0.0  1.0  0.0   0.0   1.0   1.0  -0.0  0.0   1.0  1.0)
ci on solution expe